In [10]:
import numpy as np
from myhdl import *

In [2]:
Q=(4,4)

In [162]:
class Gen():
    def __init__(self, Q, N):
        self.Q=Q; self.N=N
        self.Qlen=self.Q[0]+self.Q[1]
        self.Qmin=0; self.Qmax=2**self.Qlen
        self.Qscale=2**self.Q[1]
        
        #storage
        self.aTV=np.zeros(0); self.aTVQ=np.zeros(0)
        self.bTV=np.zeros(0); self.bTVQ=np.zeros(0)
        self.cK=np.zeros(0); self.cKQ=np.zeros(0)
    
    def Genrator(self):
        np.random.seed(np.random.randint(self.Qlen))
        self.V1=np.array((1/np.random.ranf())).round(decimals=self.Q[1])
        self.V1Q=(self.V1*self.Qscale).astype(int)
        
        np.random.seed(np.random.randint(self.Qlen))
        self.V2=np.array((1/np.random.ranf())).round(decimals=self.Q[1])
        self.V2Q=(self.V2*self.Qscale).astype(int)
    
    def GenratorCheckAndAdd(self):
        if self.V1Q<self.Qmax and self.V2Q<self.Qmax:
            self.check=self.V1Q+self.V2Q-1
            self.V1pV2=(self.V1+self.V2).round(decimals=self.Q[1])
            self.V1pV2Q=(self.V1pV2*self.Qscale).astype(int)
            if self.check<self.Qmax:
                self.aTV=np.append(self.aTV, self.V1); self.aTVQ=np.append(self.aTVQ, self.V1Q).astype(int)
                self.bTV=np.append(self.bTV, self.V2); self.bTVQ=np.append(self.bTVQ, self.V1Q).astype(int)
                self.cK=np.append(self.cK, self.V1pV2); self.cKQ=np.append(self.cKQ, self.V1pV2Q).astype(int)
    
    def MakeTVs(self):
         while len(self.aTV)<=self.N:
            self.Genrator()
            self.GenratorCheckAndAdd()
        
                
        
        
            


In [163]:
T=Gen(Q, 10)
T.Q, T.N
T.Qlen
T.Qmin, T.Qmax
T.Qscale
T.Genrator()
T.V1, T.V1Q
T.V2, T.V2Q
T.Qmax, T.GenratorCheckAndAdd()

(256, None)

In [164]:
T.MakeTVs()
T.aTVQ, T.bTVQ, T.cKQ

(array([ 38, 209,  16,  38, 209,  16,  38, 209,  16,  38, 209]),
 array([ 38, 209,  16,  38, 209,  16,  38, 209,  16,  38, 209]),
 array([ 54, 248, 226,  54, 248, 226,  54, 248, 226,  54, 248]))

In [ ]:
class AddPosTVGen():
    """
    Class to generate postive random numbers to be Qed for testing 
    """
    def __init__(self, Q, N):
        """
        Take in arguments and create output holds
        Args:
            Q(tuple): Q notation tuple where Q[0] is int bit len and Q[1] is
            dec bit len
            N(int): number of values to generate
        """
        self.Q=Q; self.N=N
        self.Qlen=self.Q[0]+self.Q[1]; self.Qmax=2**self.Qlen
        self.Qscale=2**self.Q[1]
        
        self.aTV=np.zeros(0); self.aTVQ=np.zeros(0)
        self.bTV=np.zeros(0); self.bTVQ=np.zeros(0)
        self.cK=np.zeros(0); self.cKQ=np.zeros(0)

    def Genrator(self):
        """
        Random Number genrator in floating point and supsequent Qed version
        """
        self.V1=np.array((1/np.random.ranf())).round(decimals=self.Q[1])
        
        #needed to force np.random to generate a differint random num
        np.random.seed(np.random.randint(self.Qmax))
        
        self.V2=np.array((1/np.random.ranf())).round(decimals=self.Q[1])
        
        self.V1Q=(self.V1*self.Qscale).astype(int)
        self.V2Q=(self.V2*self.Qscale).astype(int)
        
    def GenratorCheckAndAdd(self):
        """
        Cheacks if the sum of the two randome numbers generated are going to break the Qmax
        if they do dont append to retrun holds
        """
        self.V1pV2=(self.V1+self.V2).round(decimals=self.Q[1])
        self.V1pV2Q=(self.V1pV2*self.Qscale).astype(int)
        if (self.V1Q+self.V2Q)<self.Qmax:
            self.aTV=np.append(self.aTV, self.V1); self.aTVQ=np.append(self.aTVQ, self.V1Q).astype(int)
            self.bTV=np.append(self.bTV, self.V1); self.bTVQ=np.append(self.bTVQ, self.V1Q).astype(int)
            self.cK=np.append(self.cK, self.V1pV2); self.cKQ=np.append(self.cKQ, self.V1pV2Q).astype(int)
    
    def MakeTVs(self):
        """
        Automates the generating, testing and appending to make the TVs
        
        Returns:
            self.aTV(np.array): floating point numbers for a
            self.aTVQ(np.array): fixed point Qed from self.aTV
            self.bTV(np.array): floating point numbers for b
            self.bTVQ(np.array): fixed point Qed from self.bTV
            self.cK(np.array): known floating point rounded sum of self.aTV, self.bTV
            self.cKQ(np.array): known fixed point Qed from self.cK

        """
        while len(self.aTV)<=self.N:
            self.Genrator()
            self.GenratorCheckAndAdd()
        #print('Done')